# Sample Foam Plant Planning

## Problem Statement
- Foam plants have complex scheduling process
- Mould arrangement on foram production line (racetracks) have numerous constraints which represent a large number of possible solutions

#### Products and Tools/Molds
- For this POC, we are considering 3 products: __Prod1, Prod2 and Prod3__
- Prod1, Prod2 and Prod3 can be made by tools/molds __Tool1, Tool2, Tool3__
- The number of tools available: __Tool1=10, Tool2=10, Tool3=10__

#### Customer Demand
- Customer demand is provided for 12 weeks which is then udpated weekly
- For this POC, we are considering it for 1 week

| Prod | Week1 |
| --- | --- |
| Prod1 | 5000 |
| Prod2 | 6000 |
| Prod3 | 3000 |

#### Carrier
- For this POC, we are considering that any Tool can be placed on any Carrier.
- One tool can be placed on one carrier.
- The complexity of FBE will be introduced at a later version.

#### Racetrack/Production Line
- Carriers are fixed to the racetrack and production lines which go around at a specific speed. For this POC, we are considering that there is 1 racetrack
- Linespeed of a Ractrack how many carriers finish the production process in a given time. For eg if it is 42 carriers per hour, it would mean 42 products were produced in an hour.
- In this POC, the speed is considered ad __42 carriers per hour__

#### Inventory and Minimum Stock
- For this POC, we will assume there is a begining inventory for this process
- Additionally, we will want that the end of the week, there should be a minimum stock for each product. This could be a derivative of the customer demand, for this POC we are defining it as an absolute value

| Prod | StartingInventory | EndingInventory |
| --- | --- | --- |
| Prod1 | 10 | 50 |
| Prod2 | 20 | 40 |
| Prod3 | 10 | 30 |

#### Factory Schedule
- Factory runs 7 days a week, 3 shifts a day of 8 hours each
- __Breaks in eah shift:__ 10 min, 20 min and 10 min
- __Production Time in each shift__ = 480 mins - 40 mins = 440 mins

- Each shift has 8 hours = __480 mins__
    
    - __Timeslot 1: 140 mins__ = 2 hours, 20 mins
    - __Break  1:  10 mins__
    
    - __Timeslot 2: 160 mins__ = 2 hours, 40 mins
    - __Break  2:  20 mins__
    
    - __Timeslot 3: 140 mins__ = 2 hours, 20 mins
    - __Break 3:   10 mins__

### Modeling

#### __Parameters__

- L = Line speed = 42 carriers per 60 mins
- R = Number of carriers on the racetrack
- D(i) = Weekly demand for part made with Tool i
- B(i) = Begining inventory for part made with Tool i
- I(i) = Ending inventory for part made with Tool i
- Q(i) = Available quantity of Tool i
- A(k) = Total time available in time slot k

__Decisision Variables__

- x(i,j,k) = 1, if Tool i is placed on Carrier j in time slot k, otherwise 0

- y(j,k) = 1, if Changeover happens at the end of Time slot k on carrier j. __This is the changeover__

#### Constraints
- The number of tools placed on the carriers is less than or equal to the available tools i in the time slot k

        Σj x(i,j,k) <= Q(i)

- The number of tools on a carrier is less than or equal to 1


        Σi x(i,j,k) <= 1

- For each product (tool), the production should be greater than or equal to demand plus ending inventory

        B(i) + L * Σk [A(k) * Σj x(i,j,k)] >= D(i) + I(i)

- The changeover value is 1 only when tool i on the carrier j changes between two timeslots. __This is the changeover__

        y[j,k] == 1, if x[i,j,k] != x[i,j,k+1]

#### Objective
- Minimize the changeover
- Optimize the schedule

### Implementation

In [1]:
import numpy as np
import pandas as pd

import gurobipy as gp
from gurobipy import GRB
from gurobipy import *

In [2]:
# Sets and Indices

# The tools are mapped directly to products
T = ['Tool1', 'Tool2', 'Tool3']

# Available quantity of Tools
Q = {'Tool1': 10, 'Tool2': 10, 'Tool3': 10}

# Demand
D = {'Tool1': 5000, 'Tool2': 6000, 'Tool3': 3000}

# Starting inventory
B = {'Tool1': 0, 'Tool2': 0, 'Tool3': 0}

# Ending inventory 
I = {'Tool1': 50, 'Tool2': 40, 'Tool3': 30}

# Line speed per minute molds produced per minute
L = 42/60

# Carriers
R = ['C1', 'C2', 'C3', 'C4', 'C5']

In [3]:
# Timeslots
# This is for a period of 7 days
# Each day has 3 shifts
# Each shift has 3 time slots or periods as in this case
K = ['D1S1P1', 'D1S1P2', 'D1S1P3', 'D1S2P1','D1S2P2','D1S2P3','D1S3P1','D1S3P2','D1S3P3',
     'D2S1P1', 'D2S1P2', 'D2S1P3', 'D2S2P1','D2S2P2','D2S2P3','D2S3P1','D2S3P2','D2S3P3',
     'D3S1P1', 'D3S1P2', 'D3S1P3', 'D3S2P1','D3S2P2','D3S2P3','D3S3P1','D3S3P2','D3S3P3',
     'D4S1P1', 'D4S1P2', 'D4S1P3', 'D4S2P1','D4S2P2','D4S2P3','D4S3P1','D4S3P2','D4S3P3',
     'D5S1P1', 'D5S1P2', 'D5S1P3', 'D5S2P1','D5S2P2','D5S2P3','D5S3P1','D5S3P2','D5S3P3',
     'D6S1P1', 'D6S1P2', 'D6S1P3', 'D6S2P1','D6S2P2','D6S2P3','D6S3P1','D6S3P2','D6S3P3',
     'D7S1P1', 'D7S1P2', 'D7S1P3', 'D7S2P1','D7S2P2','D7S2P3','D7S3P1','D7S3P2','D7S3P3'
    ]

In [4]:
# Timeslot durations: Period 1 has 140 mins, Period 2 has 160 mins, Period 3 has 140 mins
A = {'D1S1P1': 140, 'D1S1P2': 160, 'D1S1P3': 140,
     'D1S2P1': 140, 'D1S2P2': 160, 'D1S2P3': 140,
     'D1S3P1': 140, 'D1S3P2': 160, 'D1S3P3': 140,
     'D2S1P1': 140, 'D2S1P2': 160, 'D2S1P3': 140,
     'D2S2P1': 140, 'D2S2P2': 160, 'D2S2P3': 140,
     'D2S3P1': 140, 'D2S3P2': 160, 'D2S3P3': 140,
     'D3S1P1': 140, 'D3S1P2': 160, 'D3S1P3': 140,
     'D3S2P1': 140, 'D3S2P2': 160, 'D3S2P3': 140,
     'D3S3P1': 140, 'D3S3P2': 160, 'D3S3P3': 140,
     'D4S1P1': 140, 'D4S1P2': 160, 'D4S1P3': 140,
     'D4S2P1': 140, 'D4S2P2': 160, 'D4S2P3': 140,
     'D4S3P1': 140, 'D4S3P2': 160, 'D4S3P3': 140,
     'D5S1P1': 140, 'D5S1P2': 160, 'D5S1P3': 140,
     'D5S2P1': 140, 'D5S2P2': 160, 'D5S2P3': 140,
     'D5S3P1': 140, 'D5S3P2': 160, 'D5S3P3': 140,
     'D6S1P1': 140, 'D6S1P2': 160, 'D6S1P3': 140,
     'D6S2P1': 140, 'D6S2P2': 160, 'D6S2P3': 140,
     'D6S3P1': 140, 'D6S3P2': 160, 'D6S3P3': 140,
     'D7S1P1': 140, 'D7S1P2': 160, 'D7S1P3': 140,
     'D7S2P1': 140, 'D7S2P2': 160, 'D7S2P3': 140,
     'D7S3P1': 140, 'D7S3P2': 160, 'D7S3P3': 140
    }

In [5]:
# Create the model
factory = gp.Model('Foam Planning')

# Add the variables

# If tool is placed on carrier in that time slot
x = factory.addVars(T, R, K, vtype=GRB.BINARY, name='x')

# If changeover happens at the end of timeslot k
y = factory.addVars(R,K, vtype=GRB.BINARY, name='y')

Using license file /Users/mayukh/gurobi.lic
Academic license - for non-commercial use only


In [6]:
# Add the constraints

# Total number of Tools on the carrier cannot be more than 
# the total number of available tools
for k in K:
    for i in T:
        factory.addConstr(gp.quicksum(x[i,j,k] for j in R) <= Q[i])

In [7]:
# The number of tools on a carrier is less than or equal to 1
# Complexity of matching and FBE will be added later
for k in K:
    for j in R:
        factory.addConstr(gp.quicksum(x[i,j,k] for i in T) <= 1)

In [8]:
# For each product (tool), the production should 
# be greater than or equal to the customer demand and the end inventory
req = factory.addConstrs((
    B[i] + L*(gp.quicksum(A[k]*(gp.quicksum(x[i,j,k] for j in R)) for k in K)) >=
    D[i] + I[i] for i in T), name='req')

In [9]:
# Changeover happens only when the tool on a carrier is different 
# on a timeslot than the prior timeslot
for j in R:
    for k in K:
        # The last one will always be 0
        if(k!=K[-1]):
            for i in T:
                # it will be set to 0 if both of them is 0 or 1, so 0-0=0 and 1-1=0
                factory.addConstr((y[j,k]==0) >> (x[i,j,k]-x[i,j,K[K.index(k)+1]]==0))
                
                # it will be set to 1 if either of them is 0 or 1, so 0+1=1
                factory.addConstr((y[j,k]==1) >> (x[i,j,k]+x[i,j,K[K.index(k)+1]]==1))


In [10]:
# Define the objectives. Both are to minimize
#factory.modelSense = GRB.OPTIMIZE

# Minimize the production to match demand plus on hand inventory
#obj1 = gp.quicksum(B[i] + L*(gp.quicksum(A[k]*(gp.quicksum(x[i,j,k] for j in R)) for k in K)) - 
#                   (D[i] + I[i]) for i in T)

# Objective 2 is to reduce the number of change overs
#obj2 = gp.quicksum(y[j,k] for j in R for k in K)

#factory.setObjective(obj1, GRB.MINIMIZE)
#, index=0, priority=1, name='obj1')
#factory.setObjectiveN(obj2, index=1, priority=2, name='obj2')

In [11]:
# Objective is to minimize the inventory for each of the tool
factory.modelSense = GRB.MINIMIZE

idx = 0
for i in T:
    obj = B[i] + L*(gp.quicksum(A[k]*(gp.quicksum(x[i,j,k] for j in R)) for k in K)) - (D[i] + I[i])
    factory.setObjectiveN(obj, index=idx, priority=1)
    idx += 1
    
# Add objective to minimize Changeovers
obj2 = gp.quicksum(y[j,k] for j in R for k in K)
factory.setObjectiveN(obj2, idx, priority=2)

In [12]:
factory.write('model.lp')

In [13]:
factory.optimize()

Gurobi Optimizer version 9.0.1 build v9.0.1rc0 (mac64)
Optimize a model with 507 rows, 1260 columns and 2835 nonzeros
Model fingerprint: 0x95ee4dc9
Model has 1860 general constraints
Variable types: 0 continuous, 1260 integer (1260 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+02]
  Objective range  [1e+00, 1e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 6e+03]

---------------------------------------------------------------------------
Multi-objectives: starting optimization with 4 objectives (2 combined) ...
---------------------------------------------------------------------------

Multi-objectives: applying initial presolve ...
---------------------------------------------------------------------------

Presolve added 5391 rows and 1860 columns
Presolve time: 0.08s
Presolved: 5898 rows and 3120 columns
---------------------------------------------------------------------------

Multi-objectives: optimize objective 1 () ...
----------------------

In [14]:
# Write the plan to a dataframe
rows = K.copy()
columns = R.copy()
prod_plan = pd.DataFrame(columns=columns, index=rows, data=0.0)
m = 1
for i, j, k in x.keys():
    if(x[i, j, k].x==1):
        prod_plan.loc[k, j]=i
prod_plan.head(10)

,C1,C2,C3,C4,C5
D1S1P1,Tool3,Tool1,0.0,0.0,Tool2
D1S1P2,Tool3,Tool1,0.0,0.0,Tool2
D1S1P3,Tool3,Tool1,0.0,0.0,Tool2
D1S2P1,Tool3,Tool1,0.0,0.0,Tool2
D1S2P2,Tool3,Tool1,0.0,0.0,Tool2
D1S2P3,Tool3,Tool1,0.0,0.0,Tool2
D1S3P1,Tool3,Tool1,0.0,0.0,Tool2
D1S3P2,Tool3,Tool1,0.0,0.0,Tool2
D1S3P3,Tool3,Tool1,0.0,0.0,Tool2
D2S1P1,Tool3,Tool1,0.0,0.0,Tool2


In [15]:
# What quantities of each Tool is produced
rows = K.copy()
columns = R.copy()
produced_qty = {'Tool1': 0, 'Tool2': 0, 'Tool3': 0}
for i, j, k in x.keys():
    if(x[i, j, k].x==1):
        produced_qty[i] += L*A[k]

print('Tool  ->\tProduced Qty\tCust Demand\tDifference')
for pq in produced_qty:
    print(pq, '->\t', produced_qty[pq], '\t', D[pq], '\t\t', produced_qty[pq]-D[pq])

Tool  ->	Produced Qty	Cust Demand	Difference
Tool1 ->	 6468.0 	 5000 		 1468.0
Tool2 ->	 6468.0 	 6000 		 468.0
Tool3 ->	 6468.0 	 3000 		 3468.0


In [16]:
# Write the plan to a dataframe
rows = K.copy()
columns = R.copy()
change_plan = pd.DataFrame(columns=columns, index=rows, data=0.0)

for j, k in y.keys():
    change_plan.loc[k, j]=y[j,k].x
change_plan.head(10)

,C1,C2,C3,C4,C5
D1S1P1,0.0,0.0,0.0,0.0,0.0
D1S1P2,0.0,0.0,0.0,0.0,0.0
D1S1P3,0.0,0.0,0.0,0.0,0.0
D1S2P1,0.0,0.0,0.0,0.0,0.0
D1S2P2,0.0,0.0,0.0,0.0,0.0
D1S2P3,0.0,0.0,0.0,0.0,0.0
D1S3P1,0.0,0.0,0.0,0.0,0.0
D1S3P2,0.0,0.0,0.0,0.0,0.0
D1S3P3,0.0,0.0,0.0,0.0,0.0
D2S1P1,0.0,0.0,0.0,0.0,0.0


In [17]:
# Export the plan to a csv file
prod_plan.to_csv('plan.csv')
change_plan.to_csv('change.csv')

In [18]:
# Export the plan to a .sol file
factory.write('b.sol')

Thank you